### **CodeAlpha Task 3 - Hafiza Aunsa AD - Generating Music**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
!pip install pretty_midi
import pretty_midi

# Generator Model
def generate_noise(batch_size, latent_dim):
    return torch.randn(batch_size, latent_dim)

class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Training Settings
latent_dim = 100
output_dim = 50  # Sequence length (e.g., 50 MIDI notes)
batch_size = 32
epochs = 5000

# Models
generator = Generator(latent_dim, output_dim)
discriminator = Discriminator(output_dim)

g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
criterion = nn.BCELoss()

# Training Loop
for epoch in range(epochs):
    # Generate Random Data
    noise = generate_noise(batch_size, latent_dim)
    fake_data = generator(noise)
    real_data = torch.randn(batch_size, output_dim)  # Placeholder for real MIDI data

    # Train Discriminator
    d_optimizer.zero_grad()
    real_preds = discriminator(real_data)
    fake_preds = discriminator(fake_data.detach())

    real_loss = criterion(real_preds, torch.ones_like(real_preds))
    fake_loss = criterion(fake_preds, torch.zeros_like(fake_preds))
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()

    # Train Generator
    g_optimizer.zero_grad()
    fake_preds = discriminator(fake_data)
    g_loss = criterion(fake_preds, torch.ones_like(fake_preds))
    g_loss.backward()
    g_optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch}: D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

# Generate MIDI File
def generate_midi(generator, filename="generated_music.mid"):
    generator.eval()
    noise = generate_noise(1, latent_dim)
    midi_data = generator(noise).detach().numpy().flatten()
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    start = 0
    for note_value in midi_data:
        pitch = int((note_value + 1) * 60)  # Scale to MIDI range
        note = pretty_midi.Note(velocity=101, pitch=pitch, start=start, end=start + 0.5)
        instrument.notes.append(note)
        start += 0.5

    midi.instruments.append(instrument)
    midi.write(filename)
    print(f"MIDI file saved as {filename}")

# Generate and Save Music
generate_midi(generator)

Epoch 0: D Loss: 1.3724843263626099, G Loss: 0.6805403232574463
Epoch 500: D Loss: 0.06574980169534683, G Loss: 3.490666151046753
Epoch 1000: D Loss: 0.007186220493167639, G Loss: 6.247939109802246
Epoch 1500: D Loss: 0.011120513081550598, G Loss: 4.86834192276001
Epoch 2000: D Loss: 0.000488320249132812, G Loss: 8.858254432678223
Epoch 2500: D Loss: 0.00014059807290323079, G Loss: 9.303260803222656
Epoch 3000: D Loss: 0.00015897653065621853, G Loss: 8.844026565551758
Epoch 3500: D Loss: 9.668324491940439e-05, G Loss: 9.401281356811523
Epoch 4000: D Loss: 0.0008126571774482727, G Loss: 9.1934814453125
Epoch 4500: D Loss: 0.00011955638183280826, G Loss: 9.065850257873535
MIDI file saved as generated_music.mid
